In [1]:
import sys
from PyQt5.QtCore import QtMsgType, QMessageLogContext, qInstallMessageHandler

def messageHandler(type_, context, message):
    #if "QObject::moveToThread: Current thread is not the object's thread" not in message:
    if "moveToThread" not in message:
        # Se a mensagem não contém o aviso que deseja desativar, imprima normalmente.
        sys.__stdout__.write(message)
    else:
        # Se a mensagem contém o aviso que deseja desativar, ignore-a.
        pass

# Substitua o manipulador de mensagens padrão do PyQt ou PySide pelo nosso manipulador personalizado.
qInstallMessageHandler(messageHandler)

In [2]:
import torch
import gc

def calculate_tensors():
    num_tensors=0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                #print(type(obj), obj.size())
                num_tensors+=1
        except:
            pass
        
    print("num_tensors: {}".format(num_tensors))

In [3]:
import torch
import numpy as np

from env import make_pytorch_env
from decision_transformer.models.decision_transformer import DecisionTransformer


In [4]:
class MyClass:
    # vars to class
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

In [5]:
#loaded_model = torch.load("./exp/2023.03.16/170104-default/model.pt")
#loaded_pretrain_model = torch.load("./exp/2023.03.16/170104-default/pretrain_model.pt")

#saved_model_name = '2023.03.20/222713'
saved_model_name = '2023.03.22/120246'
loaded_model = torch.load("./exp/{}-default/model.pt".format(saved_model_name))


variant = loaded_model['args']
args = MyClass(**variant)


In [6]:
loaded_model.keys()

dict_keys(['model_state_dict', 'optimizer_state_dict', 'scheduler_state_dict', 'pretrain_iter', 'online_iter', 'args', 'total_transitions_sampled', 'np', 'python', 'pytorch', 'log_temperature_optimizer_state_dict'])

In [7]:
def _get_env_spec(variant):
        #####env = gym.make(variant["env"])
        env = make_pytorch_env(args)
        state_dim = env.observation_space.shape[0]
        act_dim = env.action_space.shape[0]
        
        action_range = [
            float(env.action_space.low.min()) + 1e-6,
            float(env.action_space.high.max()) - 1e-6,
        ]
        
        env.close()
        return state_dim, act_dim, action_range

In [8]:
state_dim, act_dim, action_range = _get_env_spec(vars(args))
target_entropy = -act_dim

MAX_EPISODE_LEN = 4000

model = DecisionTransformer(
            state_dim=state_dim,
            act_dim=act_dim,
            action_range=action_range,
            max_length=variant["K"],
            eval_context_length=variant["eval_context_length"],
            max_ep_len=MAX_EPISODE_LEN,
            hidden_size=variant["embed_dim"],
            n_layer=variant["n_layer"],
            n_head=variant["n_head"],
            n_inner=4 * variant["embed_dim"],
            activation_function=variant["activation_function"],
            n_positions=1024,
            resid_pdrop=variant["dropout"],
            attn_pdrop=variant["dropout"],
            stochastic_policy=True,
            ordering=variant["ordering"],
            init_temperature=variant["init_temperature"],
            target_entropy=target_entropy,
        ).to(device=args.device)
        
        

/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


## Rascunhos

In [9]:
import pickle

In [10]:
#TODO: save state_mean, state_std

def _load_dataset(env_name):

        dataset_path = f"./data/{env_name}.pkl"
        with open(dataset_path, "rb") as f:
            trajectories = pickle.load(f)

        states, traj_lens, returns = [], [], []
        for path in trajectories:
            states.append(path["observations"])
            traj_lens.append(len(path["observations"]))
            returns.append(path["rewards"].sum())
        traj_lens, returns = np.array(traj_lens), np.array(returns)

        # used for input normalization
        states = np.concatenate(states, axis=0)
        state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        num_timesteps = sum(traj_lens)

        print("=" * 50)
        print(f"Starting new experiment: {env_name}")
        print(f"{len(traj_lens)} trajectories, {num_timesteps} timesteps found")
        print(f"Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}")
        print(f"Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}")
        print(f"Average length: {np.mean(traj_lens):.2f}, std: {np.std(traj_lens):.2f}")
        print(f"Max length: {np.max(traj_lens):.2f}, min: {np.min(traj_lens):.2f}")
        print("=" * 50)

        sorted_inds = np.argsort(returns)  # lowest to highest
        num_trajectories = 1
        timesteps = traj_lens[sorted_inds[-1]]
        ind = len(trajectories) - 2
        while ind >= 0 and timesteps + traj_lens[sorted_inds[ind]] < num_timesteps:
            timesteps += traj_lens[sorted_inds[ind]]
            num_trajectories += 1
            ind -= 1
        sorted_inds = sorted_inds[-num_trajectories:]
        trajectories = [trajectories[ii] for ii in sorted_inds]

        return trajectories, state_mean, state_std

In [11]:
device=args.device
torch.no_grad()

#Load the weights on the model
model.load_state_dict(loaded_model['model_state_dict'])
# Set model to evaluation mode
model.eval()
# Convert model to GPU
model.to(device=args.device);

In [12]:
# Nao gostei disso pq tem a ver com o Dataset
offline_trajs, state_mean, state_std = _load_dataset(args.env)
state_mean = torch.from_numpy(state_mean).to(device=device)
state_std = torch.from_numpy(state_std).to(device=device)

Starting new experiment: drone_dataset
1254 trajectories, 2078106 timesteps found
Average return: 3642.12, std: 956.57
Max return: 5216.00, min: 636.07
Average length: 1657.18, std: 468.56
Max length: 3177.00, min: 920.00


In [13]:
vec_env = make_pytorch_env(args)

In [14]:
num_envs = 1
reward_scale = 1.0 if "antmaze" in variant["env"] else 0.001

#max_ep_len = 4000
max_ep_len = 2000 # WARNING! JUST FOR DEBBUGING
vec_env.max_step = max_ep_len
num_test_episodes = 20
use_mean = True # False # True
mode = 'normal' # delayed
vec_env.cv2_show_render = False  # True

In [15]:
for _ in range(num_test_episodes):

    ###
    state = vec_env.reset()
    unfinished = np.ones(num_envs).astype(bool)
    # Not sure:
    target_return = [variant['eval_rtg'] * reward_scale] * num_envs

    ep_return = target_return

    target_return = torch.tensor(ep_return, device=device, dtype=torch.float32).reshape(
        num_envs, -1, 1
    )
    timesteps = torch.tensor([0] * num_envs, device=device, dtype=torch.long).reshape(
        num_envs, -1
    )


    states = (
        torch.from_numpy(state)
        .reshape(num_envs, state_dim)
        .to(device=device, dtype=torch.float32)
    ).reshape(num_envs, -1, state_dim)

    actions = torch.zeros(0, device=device, dtype=torch.float32)

    rewards = torch.zeros(0, device=device, dtype=torch.float32)

    ep_return = target_return
    target_return = torch.tensor(ep_return, device=device, dtype=torch.float32).reshape(
        num_envs, -1, 1
    )
    timesteps = torch.tensor([0] * num_envs, device=device, dtype=torch.long).reshape(
        num_envs, -1
    )

    # episode_return, episode_length = 0.0, 0
    episode_return = np.zeros((num_envs, 1)).astype(float)
    episode_length = np.full(num_envs, np.inf)

    # TODO: read the paper and figure it out if reward state is necessary

    ###

    with torch.no_grad():

        for t in range(max_ep_len):
            # add padding
            actions = torch.cat(
                [
                    actions,
                    torch.zeros((num_envs, act_dim), device=device).reshape(
                        num_envs, -1, act_dim
                    ),
                ],
                dim=1,
            )
            rewards = torch.cat(
                [
                    rewards,
                    torch.zeros((num_envs, 1), device=device).reshape(num_envs, -1, 1),
                ],
                dim=1,
            )

            state_pred, action_dist, reward_pred = model.get_predictions(
                (states.to(dtype=torch.float32) - state_mean) / state_std,
                actions.to(dtype=torch.float32),
                rewards.to(dtype=torch.float32),
                target_return.to(dtype=torch.float32),
                timesteps.to(dtype=torch.long),
                num_envs=num_envs,
            )
            state_pred = state_pred.detach().cpu().numpy().reshape(num_envs, -1)
            reward_pred = reward_pred.detach().cpu().numpy().reshape(num_envs)

            # the return action is a SquashNormal distribution
            action = action_dist.sample().reshape(num_envs, -1, act_dim)[:, -1]
            if use_mean:
                action = action_dist.mean.reshape(num_envs, -1, act_dim)[:, -1]
            action = action.clamp(*model.action_range)

            # TODO: nao entendo pq esta gerando um [] a mais e se isso atrapalhou no training
            #print("action: {}".format(action[0]))
            state, reward, done, _ = vec_env.step(action.detach().cpu().numpy()[0])
            vec_env.render()
                
            #state, reward, done, _ = vec_env.step(action.detach().cpu().numpy())

            # eval_env.step() will execute the action for all the sub-envs, for those where
            # the episodes have terminated, the envs will be reset. Hence we use
            # "unfinished" to track whether the first episode we roll out for each sub-env is
            # finished. In contrast, "done" only relates to the current episode
            # TODO: nao sei pq, mas o unfinished precisa por [0]
            episode_return[unfinished] += reward[unfinished[0]].reshape(-1, 1)
            #episode_return[unfinished] += reward[unfinished[0]].reshape(-1, 1)

            actions[:, -1] = action
            state = (
                torch.from_numpy(state).to(device=device).reshape(num_envs, -1, state_dim)
            )
            states = torch.cat([states, state], dim=1)
            #print("states: {}".format(states))
            # TODO: n sei pq, mas tive que por np.array em reward (na vdd sei, apenas 1 evaluate..)
            reward = torch.from_numpy(np.array(reward)).to(device=device).reshape(num_envs, 1)
            #reward = torch.from_numpy(reward).to(device=device).reshape(num_envs, 1)
            rewards[:, -1] = reward

            if mode != "delayed":
                pred_return = target_return[:, -1] - (reward * reward_scale)
            else:
                pred_return = target_return[:, -1]
            target_return = torch.cat(
                [target_return, pred_return.reshape(num_envs, -1, 1)], dim=1
            )

            timesteps = torch.cat(
                [
                    timesteps,
                    torch.ones((num_envs, 1), device=device, dtype=torch.long).reshape(
                        num_envs, 1
                    )
                    * (t + 1),
                ],
                dim=1,
            )
            
            # TODO: I comment this below, but I think it is important
            '''
            if t == max_ep_len - 1:
                done = np.ones(done.shape).astype(bool)
            if np.any(done):
                ind = np.where(done)[0]
                unfinished[ind] = False
                episode_length[ind] = np.minimum(episode_length[ind], t + 1)

            if not np.any(unfinished):
                break
            '''
        print("Episode Return: {}".format(episode_return[0][0]))
        #calculate_tensors()

/tmp/ipykernel_27543/4214266045.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_return = torch.tensor(ep_return, device=device, dtype=torch.float32).reshape(


saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_1_sum_r_-9104.73.jpg
Episode Return: -9104.729969142962
saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_2_sum_r_-10948.75.jpg
Episode Return: -10948.746212501861
saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_3_sum_r_-10306.03.jpg
Episode Return: -10306.028236458274
saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_4_sum_r_-739.43.jpg
Episode Return: -739.4250662309463
saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_5_sum_r_-8629.00.jpg
Episode Return: -8629.001066335715
saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_6_sum_r_-5714.00.jpg
Episode Return: -5714.0
saving results to: results/testing_images/screen_f_decay_f_decay_norm_flew_map_h_min_7_sum_r_-6527.29.jpg
Episode Return: -6527.29220747211
saving results to: res

In [16]:
gc.get_objects()

[[-10616.82823417156],
 tensor([3.6000], device='cuda:0'),
 tensor([[[3.6000]]], device='cuda:0'),
 tensor([[[-0.0259,  0.0134,  0.3043],
          [-0.0097,  0.0047,  0.1900],
          [-0.0021, -0.0066,  0.0651],
          ...,
          [ 0.0096,  0.0373,  0.0010],
          [-0.0017,  0.0252,  0.0017],
          [ 0.0474, -0.1636,  0.0034]]], device='cuda:0'),
 tensor([[[    6.0000],
          [   -9.0000],
          [   -5.0000],
          ...,
          [  -12.0000],
          [   -2.0000],
          [-1852.4141]]], device='cuda:0'),
 tensor([[[ 6.0840e-03,  9.2802e-02, -3.4516e-04],
          [-8.1053e-02, -2.1534e-03,  3.9034e-03],
          [-8.9367e-02, -1.6910e-03,  6.3237e-03],
          [-9.2218e-02,  9.1575e-03,  7.7062e-03],
          [-9.2976e-02,  1.5980e-02,  8.5336e-03],
          [-9.3033e-02,  2.3348e-02,  9.0236e-03],
          [-9.3736e-02,  2.8769e-02,  9.3547e-03],
          [-9.4397e-02,  3.1919e-02,  9.5733e-03],
          [-9.4315e-02,  3.3892e-02,  9.7146e

In [17]:
calculate_tensors()

num_tensors: 301


/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/torch/distributed/distributed_c10d.py:262: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/scipy/_lib/_util.py:202: DeprecationWarning: Module scipy.linalg.blas.clapack is deprecated, use scipy.linalg.lapack instead
  warnings.warn("Module %s is deprecated, use %s instead"
/home/gabriel/miniconda3/envs/odt/lib/python3.8/site-packages/scipy/_lib/_util.py:202: DeprecationWarning: Module scipy.linalg.blas.flapack is deprecated, use scipy.linalg.lapack instead
  warnings.warn("Module %s is deprecated, use %s instead"


In [18]:
vec_env.close()

In [19]:
states.shape

torch.Size([1, 2001, 4])

## Fim dos Rascunhos

In [20]:
a

NameError: name 'a' is not defined

In [ ]:
env.reset()
env.close()

terminal = False

while not terminal:
    
    
    with torch.no_grad():
        action = model(env.state) # mode.get_predictions blah
    
    #action = np.array([3.0,3.0,3.0])
    next_state, reward, terminal, _ = env.step(action)
    env.render()
